In [14]:
import pandas as pd
import plotly.express as px
import json

# Set the Mapbox access token
token = "pk.eyJ1Ijoiam9obm55amh1IiwiYSI6ImNsdG5ibzR3MzA0eHkycW83emZzdTgzNDEifQ.a67e8S73N6kNgSvFg7EMGg"
px.set_mapbox_access_token(token)

# Reading CSV into DataFrame objects
sites_df = pd.read_csv('sites_with_box_and_coords.csv')
sites_df = sites_df[sites_df["Address"].notna()]

svi_df = pd.read_csv("Maryland.csv")
svi_df = svi_df[(svi_df['STATE'] == 'Maryland') & (svi_df['COUNTY'] == 'Baltimore City')].reset_index(drop=True)

with open('Bounds.geojson') as f:
    tracts = json.load(f)

filtered_features = [feature for feature in tracts["features"] if feature["properties"]["COUNTYFP20"] == '510']
tracts["features"] = filtered_features

groceryStores = pd.read_csv("Grocery_Stores.csv")



# Create the choropleth map
fig = px.choropleth_mapbox(svi_df, geojson=tracts, locations='FIPS', featureidkey='properties.GEOID20', color='RPL_THEMES',
                           color_continuous_scale='oryel',
                           range_color=(0, 1),
                           mapbox_style="light",
                            zoom=10,
                          center={"lat": 39.29, "lon": -76.62},
                           opacity=0.8,
                           hover_data=['COUNTY', "RPL_THEMES", "EP_NOVEH"],
                           labels={'RPL_THEMES': 'SVI National Percentile', 'EP_NOVEH': '% Household without vehicles'},
                           width=600,
                           height=600
                          )

fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 100},
    title="Distribution Site and Grocery Stores against Social Vulnerability"
)

fig.update_traces(marker_line_width=0.1)

# Update layout for color bar
fig.update_layout(
    coloraxis_colorbar=dict(
        title='',
        orientation='h',  # 'h' for horizontal
        x=0.5,  # Center align the color bar on the x-axis
        xanchor='center',  # Anchor the color bar at the center
        y=-0.2,  # Position the color bar slightly below the plot area
        yanchor='bottom'  # Anchor the color bar at the bottom
    ),
    annotations=[
        dict(
            text="Social Vulnerability Index Percentile",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=-0.08,
            xanchor="center",
            yanchor="bottom",
            align="center",
            font=dict(size=14)
        )
    ]
)

# Distribution sites
sites_df['dot_size'] = (sites_df['Running_Total'] / 800)
siteTrace = px.scatter_mapbox(
    sites_df,
    lat='lat',
    lon='lon',
    size='dot_size',
    opacity=0.75,
    hover_name="Sites",
    hover_data={
        'lat': False,
        'lon': False,
        'Address': True,
        'dot_size': False,
        'Average per Month': True,
        'Running_Total': True
    }
)

siteTrace.update_traces(marker=dict(color='#59d48a', sizemode='diameter', sizeref=0.35))
fig.add_trace(siteTrace.data[0])

# Grocery stores
groceryTrace = px.scatter_mapbox(
    groceryStores,
    lat='latitude',
    lon='longitude',
    text='address',
    opacity=1,
    hover_name='storename'
)

groceryTrace.update_traces(mode="markers", marker=dict(color='#0d46b8', size=12, symbol="grocery"))
fig.add_trace(groceryTrace.data[0])

# Display the figure
fig.show()

# Save the figure as HTML
fig.write_html("svi_dist_sites.html")